Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

Loading data that has been downloaded before, there will be 3 set of datas, train data is used to train logistic model, validation data will be used to validate the train result of every stage and test data will be used to test the classification accuracy.

In [4]:
with open('notMNIST.pickle', 'rb') as f:
    data_set = pickle.load(f)
image_size = 28
num_features = image_size * image_size
train_size = 6400
valid_size = 1000
test_size = 1000
train_dataset = np.reshape(data_set['train_dataset'], (-1, num_features))[:train_size,:]
train_label = data_set['train_labels'][:train_size]
valid_dataset = data_set['valid_dataset'].reshape(valid_size, num_features)[:valid_size,:]
valid_label = data_set['valid_labels'][:valid_size]
test_dataset = data_set['test_dataset'].reshape(test_size, num_features)[:test_size,:]
test_label = data_set['test_labels'][:test_size]
num_classes = 10

# turn our y label from index in 1 to 10 into array of R1x10 for example turn 2 into [0,0,1,0,0,0,0,0,0,0]
def index_to_classes(y, num_classes):
    m=y.shape[0]
    Y=np.zeros((m, num_classes))
    for i in range(num_classes):
        Y[:,i]=np.array(y==i,dtype=int)
    return Y
Y_train = index_to_classes(train_label, 10)
print("train_dataset size:")
print(np.shape(train_dataset))
print("train label size:")
print(np.shape(Y_train))

train_dataset size:
(6400, 784)
train label size:
(6400, 10)


Lets init our W variables and b bias with random

In [5]:
def softMax(logits):
    # prevent exp overflow
    logits = logits / np.max(logits) * 10
    return np.exp(logits) / np.reshape(np.sum(np.exp(logits),1), (np.shape(logits)[0],1))
def LossFuction(X,W,b,Y):
    m = X.shape[0]
    p = softMax(np.dot(X,W) + b)
    L = -np.sum(Y * np.log(p)) / m
    db = -np.sum(Y * (Y - p), 0) / m 
    dW = -np.dot(X.T, Y * (Y - p)) / m
    return L, dW, db
def predict(X,W,b):
    return np.argmax(softMax(np.dot(X, W) + b), axis=1)
def accuracy(predict,y):
    return np.sum(np.array(predict == y, dtype=int)) * 100 / np.shape(y)[0] 

In [6]:
W = np.random.random((num_features,num_classes))
b = np.random.random((1,num_classes))
loop_count = 10000
batch_size = 64
index = 0
alpha = 0.01
for i in range(loop_count):
    index = (batch_size * i) % train_size
    batch_data = train_dataset[index:index+batch_size,:]
    batch_label = Y_train[index:index+batch_size]
    L,dW,db = LossFuction(batch_data,W,b,batch_label)
    if i % (loop_count // 10) == 0:
        print("The he Loss is:",L)
        print("Validate accuracy is: {}%".format(accuracy(predict(valid_dataset,W,b),valid_label)))
    W = W - alpha * dW
    b = b - alpha * db
print("Test accuracy is: {}%".format(accuracy(predict(test_dataset,W,b),test_label)))
predicts = predict(test_dataset,W,b)

for id, iv in enumerate(predicts[:20]):
    print("The letter is {}, you predict {}".format(chr(65 + test_label[id]), chr(65 + iv)))

The he Loss is: 2.33611444772
Validate accuracy is: 9%
The he Loss is: 1.48574822115
Validate accuracy is: 66%
The he Loss is: 1.17612849377
Validate accuracy is: 71%
The he Loss is: 1.0395270731
Validate accuracy is: 72%
The he Loss is: 0.973879916637
Validate accuracy is: 73%
The he Loss is: 0.949316017993
Validate accuracy is: 74%
The he Loss is: 0.93545425845
Validate accuracy is: 75%
The he Loss is: 0.925988621297
Validate accuracy is: 75%
The he Loss is: 0.919612448768
Validate accuracy is: 75%
The he Loss is: 0.916183384452
Validate accuracy is: 75%
Test accuracy is: 81%
The letter is I, you predict I
The letter is C, you predict C
The letter is I, you predict J
The letter is D, you predict E
The letter is B, you predict E
The letter is F, you predict F
The letter is E, you predict E
The letter is B, you predict B
The letter is D, you predict D
The letter is F, you predict F
The letter is G, you predict G
The letter is E, you predict E
The letter is C, you predict C
The letter i